# Radiomics csv extraction
Create csv file by fraction (or delta-fraction) from 'filtered_features_gtv.xlsx'

In [1]:
import pandas as pd 
import csv
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
import os 
import tqdm

# Initializations

## Load csv

In [98]:
patient_file = "/root/Documents/code/delta-rad/filtered_features_gtv.xlsx"
filtered_features_gtv_df = pd.read_excel(patient_file, index_col=0)
filtered_features_gtv_df.head()

,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,...,original_ngtdm_Coarseness,original_ngtdm_Contrast,Patient Rxmic,Fraction,Volume,Récidive Locale,Récidive Méta,Décès,Récidive,Évènement
ID,,,,,,,,,,,,,,,,,,,,,
Patient38_IRM_simu_mridian_gtv,0.431793,0.370555,24.678556,66.598953,53.068955,57.813322,47.606047,68.904864,24300.18231,28.756943,...,0.001856,0.175404,Patient 38,Simulation mridian,GTV,0,0,1,0,1
Patient38_mridian_ttt_2_gtv,0.442144,0.371772,24.636047,66.266497,52.185463,57.813322,47.606047,69.633631,24369.47426,29.299306,...,0.001997,0.128942,Patient 38,2,GTV,0,0,1,0,1
Patient38_mridian_ttt_4_gtv,0.442374,0.371341,24.625888,66.316118,53.068955,57.813322,47.606047,68.904864,24464.75070,29.336497,...,0.002107,0.153118,Patient 38,4,GTV,0,0,1,0,1
Patient38_mridian_ttt_5_gtv,0.442374,0.371341,24.625888,66.316118,53.068955,57.813322,47.606047,68.904864,24464.75070,29.336497,...,0.002032,0.169598,Patient 38,5,GTV,0,0,1,0,1
Patient10_IRM_simu_GIE_gtv,0.672158,0.549602,25.401274,46.217551,46.704549,54.477126,38.744658,54.865907,18814.74989,31.065518,...,0.003128,0.828998,Patient 10,Simulation GIE,GTV,0,1,1,1,1


## Get patient list

In [99]:
patient_list = np.unique(filtered_features_gtv_df['Patient Rxmic'].values)
print(patient_list.shape)

(83,)


## Get features list

In [100]:
features = list(filtered_features_gtv_df.columns)[:-8]
features 


['original_shape_Elongation',
 'original_shape_Flatness',
 'original_shape_LeastAxisLength',
 'original_shape_MajorAxisLength',
 'original_shape_Maximum2DDiameterColumn',
 'original_shape_Maximum2DDiameterRow',
 'original_shape_Maximum2DDiameterSlice',
 'original_shape_Maximum3DDiameter',
 'original_shape_MeshVolume',
 'original_shape_MinorAxisLength',
 'original_shape_Sphericity',
 'original_shape_SurfaceArea',
 'original_shape_SurfaceVolumeRatio',
 'original_shape_VoxelVolume',
 'original_firstorder_10Percentile',
 'original_firstorder_90Percentile',
 'original_firstorder_Energy',
 'original_firstorder_Entropy',
 'original_firstorder_InterquartileRange',
 'original_firstorder_Kurtosis',
 'original_firstorder_Maximum',
 'original_firstorder_MeanAbsoluteDeviation',
 'original_firstorder_Mean',
 'original_firstorder_Median',
 'original_firstorder_Range',
 'original_firstorder_RobustMeanAbsoluteDeviation',
 'original_firstorder_RootMeanSquared',
 'original_firstorder_Skewness',
 'origina

In [47]:
print(len(features))

81


3 patients were rules out 

In [63]:
def extract_rad(extracted_feat_df, fraction: str, patient_list: list, features_list: list): 
    data = []
    error_patients = set()
    for patient in patient_list: 
        row_data = []
        try:
            line = extracted_feat_df.loc[(extracted_feat_df['Patient Rxmic'] == patient) & (extracted_feat_df['Fraction'] == fraction)].iloc[0]
            for feature in features_list:
                feature_value = line[feature]
                row_data.append(feature_value) # add the value to raw_data 
        except IndexError:
            for feature in features_list: 
                row_data.append(None)
            error_patients.add(patient)
            print("Error with patient ", patient)

        data.append(row_data)

    return data, error_patients

# Get features

## Simu features

In [64]:
data_simu, error_patients = extract_rad(filtered_features_gtv_df, fraction='Simulation mridian', patient_list=patient_list, features_list=features)

Error with patient  Patient 17
Error with patient  Patient 18
Error with patient  Patient 19
Error with patient  Patient 21
Error with patient  Patient 22
Error with patient  Patient 23
Error with patient  Patient 24
Error with patient  Patient 32
Error with patient  Patient 57
Error with patient  Patient 66


In [66]:
simu_gtv_df = pd.DataFrame(data_simu, index=patient_list, columns=features)
simu_gtv_df.head()

,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,...,original_gldm_DependenceNonUniformity,original_gldm_DependenceNonUniformityNormalized,original_gldm_DependenceVariance,original_gldm_GrayLevelNonUniformity,original_gldm_LargeDependenceEmphasis,original_gldm_LargeDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Contrast
Patient 1,0.809098,0.540257,32.780488,60.675736,68.556955,69.193452,67.797028,77.197323,56816.33303,49.092636,...,2257.463629,0.171410,4.637846,465.737737,15.078056,6698.875171,0.308881,1.301331,0.001009,0.212588
Patient 10,0.579832,0.477183,25.195758,52.801040,52.083538,56.582369,36.629694,56.956781,21011.88221,30.615753,...,947.524033,0.193807,2.838401,177.880753,11.413991,4109.477194,0.311449,0.542220,0.002563,0.167525
Patient 11,0.610618,0.589149,36.211498,61.464026,63.297750,69.995168,49.332752,76.679330,51006.63593,37.531020,...,1279.819919,0.108203,8.149638,1027.238079,31.195807,5265.488502,0.170264,1.678677,0.000794,0.037831
Patient 12,0.670671,0.593019,28.770337,48.515066,52.666912,53.962602,39.693094,54.865907,29645.58725,32.537632,...,961.939090,0.139837,4.871202,379.236953,19.719291,3075.038668,0.217825,1.131467,0.001644,0.093174
Patient 13,0.875893,0.645176,31.084952,48.180582,55.443965,60.133526,49.494058,60.683304,41996.87769,42.201017,...,1690.589975,0.173643,3.605871,419.164955,13.732334,3086.406738,0.290250,1.174338,0.001131,0.121467


In [67]:
simu_gtv_df.to_csv('extracted_radiomics/simu_gtv.csv')

## F1 features

In [68]:
data_f1, error_patients = extract_rad(filtered_features_gtv_df, fraction='1', patient_list=patient_list, features_list=features)

Error with patient  Patient 38
Error with patient  Patient 80


In [69]:
f1_gtv_df = pd.DataFrame(data_f1, index=patient_list, columns=features)
f1_gtv_df.to_csv('extracted_radiomics/f1_gtv.csv')

## F2 features

In [70]:
data_f2, error_patients = extract_rad(filtered_features_gtv_df, fraction='2', patient_list=patient_list, features_list=features)

Error with patient  Patient 62
Error with patient  Patient 74
Error with patient  Patient 80
Error with patient  Patient 82
Error with patient  Patient 84
Error with patient  Patient 85


In [71]:
f2_gtv_df = pd.DataFrame(data_f2, index=patient_list, columns=features)
f2_gtv_df.to_csv('extracted_radiomics/f2_gtv.csv')

## F3 features

In [72]:
data_f3, error_patients = extract_rad(filtered_features_gtv_df, fraction='3', patient_list=patient_list, features_list=features)

Error with patient  Patient 38
Error with patient  Patient 56
Error with patient  Patient 63
Error with patient  Patient 74


In [73]:
f3_gtv_df = pd.DataFrame(data_f3, index=patient_list, columns=features)
f3_gtv_df.to_csv('extracted_radiomics/f3_gtv.csv')

# Get delta-rad features

In [103]:
def extract_delta_rad(extracted_feat_df, fraction1: str, fraction2: str, patient_list: list, features_list: list): 
    '''Compute the delta radiomics features by computing ratio between two fractions. '''

    data = []
    error_patients = set()
    for patient in patient_list: 
        row_data = []
        try:
            line1 = extracted_feat_df.loc[(extracted_feat_df['Patient Rxmic'] == patient) & (extracted_feat_df['Fraction'] == fraction1)].iloc[0]
            line2 = extracted_feat_df.loc[(extracted_feat_df['Patient Rxmic'] == patient) & (extracted_feat_df['Fraction'] == fraction2)].iloc[0]
            for feature in features_list:
                feature_value1 = line1[feature]
                feature_value2 = line2[feature]
                delta_feature = feature_value2 / feature_value1 
                row_data.append(delta_feature) # add the value to raw_data 
        except IndexError:
            for feature in features_list: 
                row_data.append(None)
            error_patients.add(patient)
            print("Error with patient ", patient)

        data.append(row_data)

    return data, error_patients

def extract_delta_rad2(extracted_feat_df, fraction1: str, fraction2: str, patient_list: list, features_list: list): 
    '''Compute the delta radiomics features by computing the relative difference between two fractions. '''
    data = []
    error_patients = set()
    for patient in patient_list: 
        row_data = []
        try:
            line1 = extracted_feat_df.loc[(extracted_feat_df['Patient Rxmic'] == patient) & (extracted_feat_df['Fraction'] == fraction1)].iloc[0]
            line2 = extracted_feat_df.loc[(extracted_feat_df['Patient Rxmic'] == patient) & (extracted_feat_df['Fraction'] == fraction2)].iloc[0]
            for feature in features_list:
                feature_value1 = line1[feature]
                feature_value2 = line2[feature]
                delta_feature = (feature_value2 - feature_value1) / feature_value1
                row_data.append(delta_feature) # add the value to raw_data 
        except IndexError:
            for feature in features_list: 
                row_data.append(None)
            error_patients.add(patient)
            print("Error with patient ", patient)

        data.append(row_data)

    return data, error_patients

## F1/F2

In [104]:
data_f1_f2, error_patients = extract_delta_rad(filtered_features_gtv_df, fraction1='1', fraction2='2', patient_list=patient_list, features_list=features)

Error with patient  Patient 38
Error with patient  Patient 62
Error with patient  Patient 74
Error with patient  Patient 80
Error with patient  Patient 82
Error with patient  Patient 84
Error with patient  Patient 85


In [106]:
f1_f2_gtv_df = pd.DataFrame(data_f1_f2, index=patient_list, columns=features)
f1_f2_gtv_df.to_csv('extracted_radiomics/f1_f2_gtv.csv')

In [107]:
# relative difference technique
data_f1_f2, error_patients = extract_delta_rad2(filtered_features_gtv_df, fraction1='1', fraction2='2', patient_list=patient_list, features_list=features)
f1_f2_gtv_df = pd.DataFrame(data_f1_f2, index=patient_list, columns=features)
f1_f2_gtv_df.to_csv('extracted_radiomics/rd_f1_f2_gtv.csv')

Error with patient  Patient 38
Error with patient  Patient 62
Error with patient  Patient 74
Error with patient  Patient 80
Error with patient  Patient 82
Error with patient  Patient 84
Error with patient  Patient 85


## F2/F3

In [78]:
data_f2_f3, error_patients = extract_delta_rad(filtered_features_gtv_df, fraction1='2', fraction2='3', patient_list=patient_list, features_list=features)

Error with patient  Patient 38
Error with patient  Patient 56
Error with patient  Patient 62
Error with patient  Patient 63
Error with patient  Patient 74
Error with patient  Patient 80
Error with patient  Patient 82
Error with patient  Patient 84
Error with patient  Patient 85


In [79]:
f2_f3_gtv_df = pd.DataFrame(data_f2_f3, index=patient_list, columns=features)
f2_f3_gtv_df.to_csv('extracted_radiomics/f2_f3_gtv.csv')

In [108]:
# relative difference technique
data_f2_f3, error_patients = extract_delta_rad2(filtered_features_gtv_df, fraction1='2', fraction2='3', patient_list=patient_list, features_list=features)
f2_f3_gtv_df = pd.DataFrame(data_f2_f3, index=patient_list, columns=features)
f2_f3_gtv_df.to_csv('extracted_radiomics/rd_f2_f3_gtv.csv')

Error with patient  Patient 38
Error with patient  Patient 56
Error with patient  Patient 62
Error with patient  Patient 63
Error with patient  Patient 74
Error with patient  Patient 80
Error with patient  Patient 82
Error with patient  Patient 84
Error with patient  Patient 85


## F1/F3

In [80]:
data_f1_f3, error_patients = extract_delta_rad(filtered_features_gtv_df, fraction1='1', fraction2='3', patient_list=patient_list, features_list=features)

Error with patient  Patient 38
Error with patient  Patient 56
Error with patient  Patient 63
Error with patient  Patient 74
Error with patient  Patient 80


In [81]:
f1_f3_gtv_df = pd.DataFrame(data_f1_f3, index=patient_list, columns=features)
f1_f3_gtv_df.to_csv('extracted_radiomics/f1_f3_gtv.csv')

In [109]:
# relative differnce technique
data_f1_f3, error_patients = extract_delta_rad2(filtered_features_gtv_df, fraction1='1', fraction2='3', patient_list=patient_list, features_list=features)
f1_f3_gtv_df = pd.DataFrame(data_f1_f3, index=patient_list, columns=features)
f1_f3_gtv_df.to_csv('extracted_radiomics/rd_f1_f3_gtv.csv')

Error with patient  Patient 38
Error with patient  Patient 56
Error with patient  Patient 63
Error with patient  Patient 74
Error with patient  Patient 80


## F1/F4

In [11]:
data_f1_f4, error_patients = extract_delta_rad(filtered_features_gtv_df, fraction1='1', fraction2='4', patient_list=patient_list, features_list=features)

Error with patient  Patient 38
Error with patient  Patient 70
Error with patient  Patient 74
Error with patient  Patient 80
Error with patient  Patient 81


In [13]:
f1_f4_gtv_df = pd.DataFrame(data_f1_f4, index=patient_list, columns=features)
f1_f4_gtv_df.to_csv('extracted_radiomics/f1_f4_gtv.csv')

In [110]:
# relative difference technique
data_f1_f4, error_patients = extract_delta_rad2(filtered_features_gtv_df, fraction1='1', fraction2='4', patient_list=patient_list, features_list=features)
f1_f4_gtv_df = pd.DataFrame(data_f1_f4, index=patient_list, columns=features)
f1_f4_gtv_df.to_csv('extracted_radiomics/rd_f1_f4_gtv.csv')

Error with patient  Patient 38
Error with patient  Patient 70
Error with patient  Patient 74
Error with patient  Patient 80
Error with patient  Patient 81


## F1/F5

In [7]:
data_f1_f5, error_patients = extract_delta_rad(filtered_features_gtv_df, fraction1='1', fraction2='5', patient_list=patient_list, features_list=features)

Error with patient  Patient 38
Error with patient  Patient 54
Error with patient  Patient 61
Error with patient  Patient 64
Error with patient  Patient 66
Error with patient  Patient 71
Error with patient  Patient 79
Error with patient  Patient 80
Error with patient  Patient 86


In [10]:
f1_f5_gtv_df = pd.DataFrame(data_f1_f5, index=patient_list, columns=features)
f1_f5_gtv_df.to_csv('extracted_radiomics/f1_f5_gtv.csv')

In [111]:
# relative difference technique
data_f1_f5, error_patients = extract_delta_rad2(filtered_features_gtv_df, fraction1='1', fraction2='5', patient_list=patient_list, features_list=features)
f1_f5_gtv_df = pd.DataFrame(data_f1_f5, index=patient_list, columns=features)
f1_f5_gtv_df.to_csv('extracted_radiomics/rd_f1_f5_gtv.csv')

Error with patient  Patient 38
Error with patient  Patient 54
Error with patient  Patient 61
Error with patient  Patient 64
Error with patient  Patient 66
Error with patient  Patient 71
Error with patient  Patient 79
Error with patient  Patient 80
Error with patient  Patient 86


# Simu - 1 mois features

## get clean df

In [70]:
# replace ' "" ' in my csv by ' " '
# No need to redo it 
# import fileinput
# import sys
# 
# with fileinput.FileInput('extractedFeatureIRM simu GIE.csv', inplace=True, backup='.bak') as file:
#     for line in file:
#         print(line.replace('""', '"'), end='')
    

In [75]:
onemth_df = pd.read_csv('extractedFeatureIRM GIE 1 mois.csv', sep=',', quotechar='"', index_col=0)
onemth_df

,Image,Mask,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,...,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
ID,,,,,,,,,,,,,,,,,,,,,
Patient48_IRM_GIE_1_mois,Data/NFcorr/Patient48_IRM_GIE_1_mois_NFc.nii,Data/Patient48/mask_dir/Patient48_IRM_GIE_1_mo...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d6af2e0833cd7c457f4fbc8724b31fdc109f7b71,...,0.007113,0.002550,0.632297,2329.648200,0.001731,0.073005,0.001045,191500.228276,0.278741,31.683907
Patient48_IRM_GIE_1_mois,Data/NFcorr/Patient48_IRM_GIE_1_mois_NFc.nii,Data/Patient48/mask_dir/Patient48_IRM_GIE_1_mo...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d6af2e0833cd7c457f4fbc8724b31fdc109f7b71,...,0.007802,0.002852,0.651101,2863.099050,0.002003,0.086754,0.000787,249589.224198,0.332439,28.008504
Patient49_IRM_GIE_1_mois,Data/NFcorr/Patient49_IRM_GIE_1_mois_NFc.nii,Data/Patient49/mask_dir/Patient49_IRM_GIE_1_mo...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},3744c679b7441d88e3cc0752ac4758a3a87e095f,...,0.022325,0.005764,0.679463,4243.941502,0.003164,0.105159,0.000934,277023.853245,0.812900,27.721896
Patient49_IRM_GIE_1_mois,Data/NFcorr/Patient49_IRM_GIE_1_mois_NFc.nii,Data/Patient49/mask_dir/Patient49_IRM_GIE_1_mo...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},3744c679b7441d88e3cc0752ac4758a3a87e095f,...,0.035909,0.006228,0.686635,5053.485274,0.003075,0.133408,0.000641,268946.983160,0.912873,16.645692
Patient50_IRM_GIE_1_mois,Data/NFcorr/Patient50_IRM_GIE_1_mois_NFc.nii,Data/Patient50/mask_dir/Patient50_IRM_GIE_1_mo...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},13adf6eab4dde14f95ba3d8bfc453aa0a1f06b60,...,0.003927,0.001023,0.579105,2661.291588,0.000675,0.059909,0.001481,109228.445450,0.305417,25.889353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Patient82_IRM_GIE_1_mois,Data/NFcorr/Patient82_IRM_GIE_1_mois_NFc.nii,Data/Patient82/mask_dir/Patient82_IRM_GIE_1_mo...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},16c096b1d0ef59153480600d92152466417d3cc2,...,0.019871,0.003956,0.597306,2328.040450,0.002292,0.153462,0.000962,50289.861379,0.533298,5.956224
Patient84_IRM_GIE_1_mois,Data/NFcorr/Patient84_IRM_GIE_1_mois_NFc.nii,Data/Patient84/mask_dir/Patient84_IRM_GIE_1_mo...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},05e3fc8604482ace4c0a3aadd16c39a319069803,...,0.004193,0.002118,0.729500,6387.536331,0.001655,0.024791,0.001991,355703.059024,0.710983,52.535329
Patient84_IRM_GIE_1_mois,Data/NFcorr/Patient84_IRM_GIE_1_mois_NFc.nii,Data/Patient84/mask_dir/Patient84_IRM_GIE_1_mo...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},05e3fc8604482ace4c0a3aadd16c39a319069803,...,0.005739,0.002632,0.730939,5883.608088,0.001898,0.016394,0.003646,308242.683420,0.898056,80.880719


In [80]:
# remove rows for which there is 'ptv' in the column 'Mask'
onemth_df = onemth_df[~onemth_df['Mask'].str.contains('ptv')]
onemth_df

,Image,Mask,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,...,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
ID,,,,,,,,,,,,,,,,,,,,,
Patient48_IRM_GIE_1_mois,Data/NFcorr/Patient48_IRM_GIE_1_mois_NFc.nii,Data/Patient48/mask_dir/Patient48_IRM_GIE_1_mo...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d6af2e0833cd7c457f4fbc8724b31fdc109f7b71,...,0.007113,0.002550,0.632297,2329.648200,0.001731,0.073005,0.001045,1.915002e+05,0.278741,31.683907
Patient49_IRM_GIE_1_mois,Data/NFcorr/Patient49_IRM_GIE_1_mois_NFc.nii,Data/Patient49/mask_dir/Patient49_IRM_GIE_1_mo...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},3744c679b7441d88e3cc0752ac4758a3a87e095f,...,0.022325,0.005764,0.679463,4243.941502,0.003164,0.105159,0.000934,2.770239e+05,0.812900,27.721896
Patient50_IRM_GIE_1_mois,Data/NFcorr/Patient50_IRM_GIE_1_mois_NFc.nii,Data/Patient50/mask_dir/Patient50_IRM_GIE_1_mo...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},13adf6eab4dde14f95ba3d8bfc453aa0a1f06b60,...,0.003927,0.001023,0.579105,2661.291588,0.000675,0.059909,0.001481,1.092284e+05,0.305417,25.889353
Patient51_IRM_GIE_1_mois,Data/NFcorr/Patient51_IRM_GIE_1_mois_NFc.nii,Data/Patient51/mask_dir/Patient51_IRM_GIE_1_mo...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},a4533d1d718914ab651d4293306d2f0b865b5b90,...,0.004217,0.001121,0.623500,3152.366263,0.000696,0.045074,0.002754,9.898376e+04,0.548338,35.714065
Patient52_IRM_GIE_1_mois,Data/NFcorr/Patient52_IRM_GIE_1_mois_NFc.nii,Data/Patient52/mask_dir/Patient52_IRM_GIE_1_mo...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},e0682288e4ba4facae5b3e2e7453a802240d0676,...,0.012993,0.003477,0.729910,5442.958909,0.002096,0.046809,0.001226,2.962116e+05,0.677814,31.052650
Patient53_IRM_GIE_1_mois,Data/NFcorr/Patient53_IRM_GIE_1_mois_NFc.nii,Data/Patient53/mask_dir/Patient53_IRM_GIE_1_mo...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},41b62439aad870364cafea0bf58cf354a57ffcb8,...,0.012682,0.004194,0.680842,4075.689863,0.003079,0.049186,0.002232,1.355213e+05,0.771258,31.013379
Patient54_IRM_GIE_1_mois,Data/NFcorr/Patient54_IRM_GIE_1_mois_NFc.nii,Data/Patient54/mask_dir/Patient54_IRM_GIE_1_mo...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},04bcd69686f1ae980ccd7a6a02dbeda53b84e5a5,...,0.129959,0.014148,0.409220,512.133918,0.004475,0.124219,0.002409,2.919827e+04,0.169196,22.827701
Patient55_IRM_GIE_1_mois,Data/NFcorr/Patient55_IRM_GIE_1_mois_NFc.nii,Data/Patient55/mask_dir/Patient55_IRM_GIE_1_mo...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},87f350262a84487b43d0b54a916459a2ce3fddfd,...,0.011200,0.003523,0.756164,6680.385654,0.002014,0.019161,0.002763,5.190645e+05,1.024692,115.931238
Patient56_IRM_GIE_1_mois,Data/NFcorr/Patient56_IRM_GIE_1_mois_NFc.nii,Data/Patient56/mask_dir/Patient56_IRM_GIE_1_mo...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},9ab7f9c6cab031ced58d0ab71deb484dbfa13da0,...,0.008506,0.002486,0.647896,3820.807912,0.001509,0.109316,0.000878,9.801073e+04,0.561293,8.547689


In [93]:
col_to_drop = 'Image,Mask,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,diagnostics_Image-original_Size,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_Hash,diagnostics_Mask-original_Spacing,diagnostics_Mask-original_Size,diagnostics_Mask-original_BoundingBox,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,diagnostics_Mask-original_CenterOfMassIndex,diagnostics_Mask-original_CenterOfMass,diagnostics_Image-interpolated_Spacing,diagnostics_Image-interpolated_Size,diagnostics_Image-interpolated_Mean,diagnostics_Image-interpolated_Minimum,diagnostics_Image-interpolated_Maximum,diagnostics_Mask-interpolated_Spacing,diagnostics_Mask-interpolated_Size,diagnostics_Mask-interpolated_BoundingBox,diagnostics_Mask-interpolated_VoxelNum,diagnostics_Mask-interpolated_VolumeNum,diagnostics_Mask-interpolated_CenterOfMassIndex,diagnostics_Mask-interpolated_CenterOfMass,diagnostics_Mask-interpolated_Mean,diagnostics_Mask-interpolated_Minimum,diagnostics_Mask-interpolated_Maximum,diagnostics_Mask-resegmented_Spacing,diagnostics_Mask-resegmented_Size,diagnostics_Mask-resegmented_BoundingBox,diagnostics_Mask-resegmented_VoxelNum,diagnostics_Mask-resegmented_VolumeNum,diagnostics_Mask-resegmented_CenterOfMassIndex,diagnostics_Mask-resegmented_CenterOfMass,diagnostics_Mask-resegmented_Mean,diagnostics_Mask-resegmented_Minimum,diagnostics_Mask-resegmented_Maximum'.split(',')

In [94]:
# keep only the column ID and the features
onemth_df = onemth_df.drop(columns=col_to_drop)
onemth_df

,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,...,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
ID,,,,,,,,,,,,,,,,,,,,,
Patient48_IRM_GIE_1_mois,0.856220,0.715668,34.207049,47.797384,59.332896,49.574515,57.026710,62.601338,43844.302180,40.925056,...,0.007113,0.002550,0.632297,2329.648200,0.001731,0.073005,0.001045,1.915002e+05,0.278741,31.683907
Patient49_IRM_GIE_1_mois,0.708011,0.429443,30.161499,70.233920,75.351147,73.927284,56.582369,80.284740,57846.509470,49.726414,...,0.022325,0.005764,0.679463,4243.941502,0.003164,0.105159,0.000934,2.770239e+05,0.812900,27.721896
Patient50_IRM_GIE_1_mois,0.515480,0.468837,29.342415,62.585530,47.016389,62.918845,47.967441,77.916728,39025.443904,32.261564,...,0.003927,0.001023,0.579105,2661.291588,0.000675,0.059909,0.001481,1.092284e+05,0.305417,25.889353
Patient51_IRM_GIE_1_mois,0.671614,0.575432,24.925757,43.316612,49.035645,49.654841,40.092699,52.893437,19566.314946,29.092046,...,0.004217,0.001121,0.623500,3152.366263,0.000696,0.045074,0.002754,9.898376e+04,0.548338,35.714065
Patient52_IRM_GIE_1_mois,0.809536,0.509697,29.714087,58.297541,55.420000,72.347032,69.633631,72.640233,43353.845082,47.193985,...,0.012993,0.003477,0.729910,5442.958909,0.002096,0.046809,0.001226,2.962116e+05,0.677814,31.052650
Patient53_IRM_GIE_1_mois,0.830732,0.604231,28.774066,47.620970,50.766061,49.868411,56.582369,58.634705,30249.004660,39.560263,...,0.012682,0.004194,0.680842,4075.689863,0.003079,0.049186,0.002232,1.355213e+05,0.771258,31.013379
Patient54_IRM_GIE_1_mois,0.784301,0.565922,24.756209,43.744945,43.858814,47.606047,45.465022,51.879087,24252.183200,34.309212,...,0.129959,0.014148,0.409220,512.133918,0.004475,0.124219,0.002409,2.919827e+04,0.169196,22.827701
Patient55_IRM_GIE_1_mois,0.700196,0.625167,23.851034,38.151467,44.040175,40.356904,38.641659,45.930149,15369.099312,26.713519,...,0.011200,0.003523,0.756164,6680.385654,0.002014,0.019161,0.002763,5.190645e+05,1.024692,115.931238
Patient56_IRM_GIE_1_mois,0.706502,0.561174,37.608657,67.017787,76.627338,69.078162,66.531161,77.386385,68235.971523,47.348218,...,0.008506,0.002486,0.647896,3820.807912,0.001509,0.109316,0.000878,9.801073e+04,0.561293,8.547689


In [95]:
# save the dataframe to a csv file
onemth_df.to_csv('extractedFeatureIRM_GIE_1month_gtv_clean.csv')

In [134]:
simu_df = pd.read_csv('extractedFeatureIRM simu GIE.csv', sep=',', quotechar='"', index_col=0)
simu_df

,Image,Mask,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,...,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
ID,,,,,,,,,,,,,,,,,,,,,
Patient1_IRM_simu_GIE,Data/NFcorr/Patient1_IRM_simu_GIE_NFc.nii,Data/Patient1/mask_dir/Patient1_IRM_simu_GIE_g...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},5f0b79b96769b8b809426b65799176b1dfe7d6e1,...,0.002371,0.001276,0.795160,19162.601719,0.001106,0.012977,0.001998,771549.182577,1.236145,106.296208
Patient1_IRM_simu_GIE,Data/NFcorr/Patient1_IRM_simu_GIE_NFc.nii,Data/Patient1/mask_dir/Patient1_IRM_simu_GIE_p...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},5f0b79b96769b8b809426b65799176b1dfe7d6e1,...,0.001943,0.001123,0.787731,16355.468127,0.000992,0.019864,0.001171,802290.461434,0.933331,67.862420
Patient2_IRM_simu_GIE,Data/NFcorr/Patient2_IRM_simu_GIE_NFc.nii,Data/Patient2/mask_dir/Patient2_IRM_simu_GIE_g...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},5ab7865d30175e813e03a6243dd73c1a1098baee,...,0.002578,0.001033,0.747700,8418.631511,0.000755,0.010430,0.005090,144838.410776,0.852376,62.352149
Patient2_IRM_simu_GIE,Data/NFcorr/Patient2_IRM_simu_GIE_NFc.nii,Data/Patient2/mask_dir/Patient2_IRM_simu_GIE_p...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},5ab7865d30175e813e03a6243dd73c1a1098baee,...,0.003449,0.001527,0.736247,8622.756404,0.001113,0.027677,0.002004,153805.578064,1.062430,24.511031
Patient3_IRM_simu_GIE,Data/NFcorr/Patient3_IRM_simu_GIE_NFc.nii,Data/Patient3/mask_dir/Patient3_IRM_simu_GIE_g...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},e5df000a573e13a44fa023ea4ffff9010ea8db7a,...,0.001656,0.000621,0.741587,10250.432129,0.000394,0.024441,0.001280,386490.628216,0.574215,36.654402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Patient83_IRM_simu_GIE,Data/NFcorr/Patient83_IRM_simu_GIE_NFc.nii,Data/Patient83/mask_dir/Patient83_IRM_simu_GIE...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},6eb3308850683100d5a61e736bffc2f334f7a079,...,0.032944,0.006375,0.665472,2878.252771,0.003197,0.085083,0.000862,261671.906741,0.393842,29.947298
Patient84_IRM_simu_GIE,Data/NFcorr/Patient84_IRM_simu_GIE_NFc.nii,Data/Patient84/mask_dir/Patient84_IRM_simu_GIE...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},0600bfd5556567847ffd4e4aa0c74d0d38143590,...,0.002864,0.001276,0.720848,5891.365569,0.000924,0.028350,0.001726,256622.024085,0.520010,37.669832
Patient84_IRM_simu_GIE,Data/NFcorr/Patient84_IRM_simu_GIE_NFc.nii,Data/Patient84/mask_dir/Patient84_IRM_simu_GIE...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},0600bfd5556567847ffd4e4aa0c74d0d38143590,...,0.002078,0.001258,0.724024,4906.379601,0.001114,0.019264,0.003514,148139.814698,0.613479,45.817758


In [135]:
simu_df = simu_df[~simu_df['Mask'].str.contains('ptv')]
simu_df

,Image,Mask,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,...,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
ID,,,,,,,,,,,,,,,,,,,,,
Patient1_IRM_simu_GIE,Data/NFcorr/Patient1_IRM_simu_GIE_NFc.nii,Data/Patient1/mask_dir/Patient1_IRM_simu_GIE_g...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},5f0b79b96769b8b809426b65799176b1dfe7d6e1,...,0.002371,0.001276,0.795160,19162.601719,0.001106,0.012977,0.001998,771549.182577,1.236145,106.296208
Patient2_IRM_simu_GIE,Data/NFcorr/Patient2_IRM_simu_GIE_NFc.nii,Data/Patient2/mask_dir/Patient2_IRM_simu_GIE_g...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},5ab7865d30175e813e03a6243dd73c1a1098baee,...,0.002578,0.001033,0.747700,8418.631511,0.000755,0.010430,0.005090,144838.410776,0.852376,62.352149
Patient3_IRM_simu_GIE,Data/NFcorr/Patient3_IRM_simu_GIE_NFc.nii,Data/Patient3/mask_dir/Patient3_IRM_simu_GIE_g...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},e5df000a573e13a44fa023ea4ffff9010ea8db7a,...,0.001656,0.000621,0.741587,10250.432129,0.000394,0.024441,0.001280,386490.628216,0.574215,36.654402
Patient4_IRM_simu_GIE,Data/NFcorr/Patient4_IRM_simu_GIE_NFc.nii,Data/Patient4/mask_dir/Patient4_IRM_simu_GIE_g...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},75893de2ed67ac104778d5692b60d5e946aa466d,...,0.001461,0.001017,0.808444,8830.026543,0.000925,0.015100,0.003885,241850.437309,1.237642,56.294698
Patient5_IRM_simu_GIE,Data/NFcorr/Patient5_IRM_simu_GIE_NFc.nii,Data/Patient5/mask_dir/Patient5_IRM_simu_GIE_g...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},56d3f08d614183d9aa8056e74aadd6484f3f7faf,...,0.001477,0.000977,0.763061,20545.770030,0.000887,0.006897,0.002752,433251.957062,0.865981,85.135525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Patient81_IRM_simu_GIE,Data/NFcorr/Patient81_IRM_simu_GIE_NFc.nii,Data/Patient81/mask_dir/Patient81_IRM_simu_GIE...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},17dad8c7772fd40a1688e190171b993acc70c331,...,1.217310,0.033761,0.180810,47.002956,0.004117,1.394178,0.001302,1526.204780,0.077862,1.297078
Patient82_IRM_simu_GIE,Data/NFcorr/Patient82_IRM_simu_GIE_NFc.nii,Data/Patient82/mask_dir/Patient82_IRM_simu_GIE...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},b5fa19f10a20042b9e325c8a246edcde4a5ec788,...,0.009581,0.003158,0.690138,3854.961112,0.002153,0.064458,0.001247,187035.439207,0.565076,25.156542
Patient83_IRM_simu_GIE,Data/NFcorr/Patient83_IRM_simu_GIE_NFc.nii,Data/Patient83/mask_dir/Patient83_IRM_simu_GIE...,v3.0.1,1.21.6,2.1.1.2,1.3.0,3.7.0,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},6eb3308850683100d5a61e736bffc2f334f7a079,...,0.034015,0.006956,0.666663,2727.158008,0.003563,0.063825,0.001355,188844.609872,0.436655,31.051768


In [136]:
# keep only the column ID and the features
simu_df = simu_df.drop(columns=col_to_drop)
simu_df

,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,...,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
ID,,,,,,,,,,,,,,,,,,,,,
Patient1_IRM_simu_GIE,0.647477,0.543005,30.124580,55.477527,48.463384,66.909465,67.797028,70.090000,32652.930141,35.920419,...,0.002371,0.001276,0.795160,19162.601719,0.001106,0.012977,0.001998,771549.182577,1.236145,106.296208
Patient2_IRM_simu_GIE,0.335118,0.279891,19.923259,71.182256,27.710000,75.509651,41.042351,75.579991,9921.380481,23.854480,...,0.002578,0.001033,0.747700,8418.631511,0.000755,0.010430,0.005090,144838.410776,0.852376,62.352149
Patient3_IRM_simu_GIE,0.850484,0.752285,34.995427,46.518866,58.430442,59.801232,53.987214,61.897090,39467.360546,39.563533,...,0.001656,0.000621,0.741587,10250.432129,0.000394,0.024441,0.001280,386490.628216,0.574215,36.654402
Patient4_IRM_simu_GIE,0.623353,0.135460,8.269632,61.048536,42.505200,58.770486,74.660406,74.731545,10308.080099,38.054778,...,0.001461,0.001017,0.808444,8830.026543,0.000925,0.015100,0.003885,241850.437309,1.237642,56.294698
Patient5_IRM_simu_GIE,0.713009,0.394110,20.655952,52.411682,45.318690,61.854151,51.183038,62.388769,20298.932980,37.369984,...,0.001477,0.000977,0.763061,20545.770030,0.000887,0.006897,0.002752,433251.957062,0.865981,85.135525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Patient81_IRM_simu_GIE,0.700755,0.478827,30.135912,62.936982,60.310000,75.351147,58.430442,79.285720,50682.551693,44.103406,...,1.217310,0.033761,0.180810,47.002956,0.004117,1.394178,0.001302,1526.204780,0.077862,1.297078
Patient82_IRM_simu_GIE,0.449658,0.364544,28.367081,77.815220,57.629203,84.146530,47.606047,85.415733,45079.467314,34.990232,...,0.009581,0.003158,0.690138,3854.961112,0.002153,0.064458,0.001247,187035.439207,0.565076,25.156542
Patient83_IRM_simu_GIE,0.471815,0.414796,27.738370,66.872315,53.814691,72.383747,46.361968,73.855370,35295.588050,31.551330,...,0.034015,0.006956,0.666663,2727.158008,0.003563,0.063825,0.001355,188844.609872,0.436655,31.051768


In [137]:
# save the dataframe to a csv file
simu_df.to_csv('extractedFeatureIRM_simu_GIE_gtv_clean.csv')

## Simu/1 mois delta-rad

In [157]:
def extract_delta_rad(simu_df, onemth_df, patient_list: list, features_list: list): 
    '''Compute the delta radiomics features by computing ratio between two fractions. '''

    data = []
    error_patients = set()
    for patient in patient_list: 
        row_data = []
        try:
            line1 = simu_df.loc[simu_df.index == patient].iloc[0]
            assert line1.shape[0] == simu_df.shape[1]
            line2 = onemth_df.loc[onemth_df.index == patient].iloc[0]
            for feature in features_list:
                feature_value1 = line1[feature]
                feature_value2 = line2[feature]
                delta_feature = feature_value2 / feature_value1 
                row_data.append(delta_feature) # add the value to raw_data 
        except IndexError:
            for feature in features_list: 
                row_data.append(None)
            error_patients.add(patient)
            print("Error with patient ", patient)

        data.append(row_data)

    return data, error_patients

def extract_delta_rad2(simu_df, onemth_df, patient_list: list, features_list: list): 
    '''Compute the delta radiomics features by computing the relative difference between two fractions. '''

    data = []
    error_patients = set()
    for patient in patient_list: 
        row_data = []
        try:
            line1 = simu_df.loc[simu_df.index == patient].iloc[0]
            assert line1.shape[0] == simu_df.shape[1]
            line2 = onemth_df.loc[onemth_df.index== patient].iloc[0]
            for feature in features_list:
                feature_value1 = line1[feature]
                feature_value2 = line2[feature]
                delta_feature = (feature_value2 - feature_value1) / feature_value1
                row_data.append(delta_feature) # add the value to raw_data 
        except IndexError:
            for feature in features_list: 
                row_data.append(None)
            error_patients.add(patient)
            print("Error with patient ", patient)

        data.append(row_data)

    return data, error_patients

In [138]:
simu_df = pd.read_csv('extractedFeatureIRM_GIE_1month_gtv_clean.csv', sep=',', index_col=0)
onemth_df = pd.read_csv('extractedFeatureIRM_simu_GIE_gtv_clean.csv', sep=',', index_col=0)


In [139]:
simu_df.head()

,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,...,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
ID,,,,,,,,,,,,,,,,,,,,,
Patient48,0.856220,0.715668,34.207049,47.797384,59.332896,49.574515,57.026710,62.601338,43844.302180,40.925056,...,0.007113,0.002550,0.632297,2329.648200,0.001731,0.073005,0.001045,191500.228276,0.278741,31.683907
Patient49,0.708011,0.429443,30.161499,70.233920,75.351147,73.927284,56.582369,80.284740,57846.509470,49.726414,...,0.022325,0.005764,0.679463,4243.941502,0.003164,0.105159,0.000934,277023.853245,0.812900,27.721896
Patient50,0.515480,0.468837,29.342415,62.585530,47.016389,62.918845,47.967441,77.916728,39025.443904,32.261564,...,0.003927,0.001023,0.579105,2661.291588,0.000675,0.059909,0.001481,109228.445450,0.305417,25.889353
Patient51,0.671614,0.575432,24.925757,43.316612,49.035645,49.654841,40.092699,52.893437,19566.314946,29.092046,...,0.004217,0.001121,0.623500,3152.366263,0.000696,0.045074,0.002754,98983.759347,0.548338,35.714065
Patient52,0.809536,0.509697,29.714087,58.297541,55.420000,72.347032,69.633631,72.640233,43353.845082,47.193985,...,0.012993,0.003477,0.729910,5442.958909,0.002096,0.046809,0.001226,296211.553476,0.677814,31.052650


In [140]:
onemth_df.head()

,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,...,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
ID,,,,,,,,,,,,,,,,,,,,,
Patient1,0.647477,0.543005,30.124580,55.477527,48.463384,66.909465,67.797028,70.090000,32652.930141,35.920419,...,0.002371,0.001276,0.795160,19162.601719,0.001106,0.012977,0.001998,771549.182577,1.236145,106.296208
Patient2,0.335118,0.279891,19.923259,71.182256,27.710000,75.509651,41.042351,75.579991,9921.380481,23.854480,...,0.002578,0.001033,0.747700,8418.631511,0.000755,0.010430,0.005090,144838.410776,0.852376,62.352149
Patient3,0.850484,0.752285,34.995427,46.518866,58.430442,59.801232,53.987214,61.897090,39467.360546,39.563533,...,0.001656,0.000621,0.741587,10250.432129,0.000394,0.024441,0.001280,386490.628216,0.574215,36.654402
Patient4,0.623353,0.135460,8.269632,61.048536,42.505200,58.770486,74.660406,74.731545,10308.080099,38.054778,...,0.001461,0.001017,0.808444,8830.026543,0.000925,0.015100,0.003885,241850.437309,1.237642,56.294698
Patient5,0.713009,0.394110,20.655952,52.411682,45.318690,61.854151,51.183038,62.388769,20298.932980,37.369984,...,0.001477,0.000977,0.763061,20545.770030,0.000887,0.006897,0.002752,433251.957062,0.865981,85.135525


In [141]:
print(simu_df.shape, onemth_df.shape)

(32, 107) (71, 107)


In [142]:
# remove patient of onemth_df that are not in simu_df
keep_patients = list(set(simu_df.index).intersection(set(onemth_df.index)))
onemth_df = onemth_df.loc[keep_patients]
simu_df = simu_df.loc[keep_patients]

print(simu_df.shape, onemth_df.shape)

(28, 107) (28, 107)


In [143]:
simu_df.to_csv('extracted_radiomics/simu_gie_gtv.csv')
onemth_df.to_csv('extracted_radiomics/gie_1month_gtv.csv')

### Ratio

In [160]:
data_simu_onemth, error_patients = extract_delta_rad(simu_df, onemth_df, patient_list=keep_patients, features_list=simu_df.columns)

In [161]:
simu_onemth_gtv_df = pd.DataFrame(data_simu_onemth, index=keep_patients, columns=simu_df.columns)
simu_onemth_gtv_df.to_csv('extracted_radiomics/simu_onemth_gtv.csv')

In [162]:
simu_onemth_gtv_df.head()

,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,...,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
Patient52,0.969905,1.000884,1.028823,1.027914,1.042356,0.979226,1.021951,1.043611,1.043965,0.996979,...,0.821388,0.854290,0.968962,0.808080,0.975675,1.329860,0.938071,0.754033,0.858702,0.820723
Patient56,0.997957,0.995115,1.007321,1.012266,1.013260,1.025564,0.975695,1.016937,1.020225,1.010198,...,2.205421,1.142098,1.079105,1.633885,0.780509,0.512187,0.897999,3.666716,0.950964,3.172720
Patient58,1.012814,1.054016,1.048623,0.994883,0.967373,1.040714,1.026045,1.004323,1.017550,1.007631,...,0.837548,1.053621,1.082088,1.893602,1.058873,0.601960,1.083378,1.663236,1.404847,1.362286
Patient54,1.003552,0.983150,0.980745,0.997554,1.000690,1.018009,1.026638,0.988582,0.981280,1.001097,...,0.715035,0.649204,1.089203,1.650302,0.714931,0.674707,0.980341,1.912857,1.198266,1.788408
Patient73,0.994921,0.978803,0.994643,1.016182,0.970919,1.030873,1.032175,1.045066,1.008969,1.011021,...,0.389488,0.701071,0.959759,0.694353,1.029682,1.469347,0.955878,0.653909,0.858413,0.692936


### Relative difference

In [163]:
data_simu_onemth, error_patients = extract_delta_rad2(simu_df, onemth_df, patient_list=keep_patients, features_list=simu_df.columns)
simu_onemth_gtv_df = pd.DataFrame(data_simu_onemth, index=keep_patients, columns=simu_df.columns)
simu_onemth_gtv_df.to_csv('extracted_radiomics/rd_simu_onemth_gtv.csv')

In [164]:
simu_onemth_gtv_df.head()

,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,...,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
Patient52,-0.030095,0.000884,0.028823,0.027914,0.042356,-0.020774,0.021951,0.043611,0.043965,-0.003021,...,-0.178612,-0.145710,-0.031038,-0.191920,-0.024325,0.329860,-0.061929,-0.245967,-0.141298,-0.179277
Patient56,-0.002043,-0.004885,0.007321,0.012266,0.013260,0.025564,-0.024305,0.016937,0.020225,0.010198,...,1.205421,0.142098,0.079105,0.633885,-0.219491,-0.487813,-0.102001,2.666716,-0.049036,2.172720
Patient58,0.012814,0.054016,0.048623,-0.005117,-0.032627,0.040714,0.026045,0.004323,0.017550,0.007631,...,-0.162452,0.053621,0.082088,0.893602,0.058873,-0.398040,0.083378,0.663236,0.404847,0.362286
Patient54,0.003552,-0.016850,-0.019255,-0.002446,0.000690,0.018009,0.026638,-0.011418,-0.018720,0.001097,...,-0.284965,-0.350796,0.089203,0.650302,-0.285069,-0.325293,-0.019659,0.912857,0.198266,0.788408
Patient73,-0.005079,-0.021197,-0.005357,0.016182,-0.029081,0.030873,0.032175,0.045066,0.008969,0.011021,...,-0.610512,-0.298929,-0.040241,-0.305647,0.029682,0.469347,-0.044122,-0.346091,-0.141587,-0.307064


# Get patient outcomes

In [3]:
outcome_features = list(filtered_features_gtv_df.columns)[-5:]
outcome_features

['Récidive Locale', 'Récidive Méta', 'Décès', 'Récidive', 'Évènement']

In [5]:
def extract_outcomes(extracted_feat_df, patient_list: list, features_list: list): 
    data = []
    error_patients = set()
    for patient in patient_list: 
        row_data = []
        try:
            line = extracted_feat_df.loc[(extracted_feat_df['Patient Rxmic'] == patient)].iloc[0]
            for feature in features_list:
                outcome_value = line[feature]
                row_data.append(outcome_value) # add the value to raw_data 
        except IndexError:
            for feature in features_list: 
                row_data.append(None)
            error_patients.add(patient)
            print("Error with patient ", patient)

        data.append(row_data)

    return data, error_patients

In [10]:
outcome_data, error_patients = extract_outcomes(filtered_features_gtv_df, patient_list, outcome_features)

In [12]:
outcome_df = pd.DataFrame(outcome_data, index=patient_list, columns=outcome_features)
outcome_df.to_csv('extracted_radiomics/outcomes.csv')

In [13]:
outcome_df

,Récidive Locale,Récidive Méta,Décès,Récidive,Évènement
Patient 1,0,1,1,1,1
Patient 10,0,1,1,1,1
Patient 11,0,1,1,1,1
Patient 12,0,0,0,0,0
Patient 13,0,1,0,1,1
...,...,...,...,...,...
Patient 83,0,0,0,0,0
Patient 84,0,1,0,1,1
Patient 85,0,0,0,0,0
Patient 86,0,0,1,0,1


## Outcomes statistics

In [14]:
outcome_df["Récidive Locale"].value_counts()

Récidive Locale
0    63
1    20
Name: count, dtype: int64

In [26]:
print('We need {} patients with Récidive locale in the test set. '.format(round(20*0.3)))

We need 6 patients with Récidive locale in the test set. 


In [15]:
outcome_df["Récidive Méta"].value_counts()

Récidive Méta
1    62
0    21
Name: count, dtype: int64

In [27]:
print('We need {} patients with Récidive Méta in the test set. '.format(int(62*0.3)))

We need 18 patients with Récidive Méta in the test set. 


In [16]:
outcome_df["Décès"].value_counts()

Décès
0    48
1    35
Name: count, dtype: int64

In [28]:
print('We need {} patients with Décès in the test set. '.format(int(35*0.3)))

We need 10 patients with Décès in the test set. 


## Outcome Simu/1 month

In [2]:
outcome_df = pd.read_csv('extracted_radiomics/outcomes.csv', index_col=0)
outcome_df

,Récidive Locale,Récidive Méta,Décès,Récidive,Évènement
Patient 1,0,1,1,1,1
Patient 10,0,1,1,1,1
Patient 11,0,1,1,1,1
Patient 12,0,0,0,0,0
Patient 13,0,1,0,1,1
...,...,...,...,...,...
Patient 83,0,0,0,0,0
Patient 84,0,1,0,1,1
Patient 85,0,0,0,0,0
Patient 86,0,0,1,0,1


In [6]:
# drop other columns than récidive locale, and décès
selected_outcome_df = outcome_df.drop(columns=['Récidive Méta', 'Évènement', 'Récidive'])
selected_outcome_df

,Récidive Locale,Décès
Patient 1,0,1
Patient 10,0,1
Patient 11,0,1
Patient 12,0,0
Patient 13,0,0
...,...,...
Patient 83,0,0
Patient 84,0,0
Patient 85,0,0
Patient 86,0,1


In [8]:
simu_onemth_gtv_df = pd.read_csv('extracted_radiomics/simu_onemth_gtv.csv', index_col=0)
selected_outcome_df = selected_outcome_df.loc[simu_onemth_gtv_df.index]
assert selected_outcome_df.shape[0] == simu_onemth_gtv_df.shape[0]

In [9]:
selected_outcome_df.to_csv('extracted_radiomics/simu_onemth_outcomes.csv')